In [225]:
import json
import string
import numpy as np

from octis.evaluation_metrics.diversity_metrics import TopicDiversity
from octis.evaluation_metrics.coherence_metrics import Coherence

In [226]:

#ds_path = '/home/timmiakov/repos/dynamic-topic-modeling/data/superseg/superseg_test_small.json'
ds_path = '/home/timmiakov/repos/dynamic-topic-modeling/data/topic_extraction/Hermes-7b-superseg-test.json'
with open(ds_path, 'r') as f:
    keywords = json.load(f)['data']

labels_path = '/home/timmiakov/repos/dynamic-topic-modeling/data/superseg/superseg_test_small.json'
with open(labels_path, 'r') as f:
    texts = json.load(f)['data']

In [227]:
keywords

[{'key_words': ['Help',
   'Planning',
   'Social Security benefits',
   'Survivors',
   'Future planning'],
  'topic_id': 0,
  'dialogue_id': 0},
 {'key_words': ['Benefits',
   'Family',
   'Pension',
   'Survivor',
   'Social Security'],
  'topic_id': 1,
  'dialogue_id': 0},
 {'key_words': ['Ex-wife', 'Benefits', 'Widow', 'Amount', 'Survivors'],
  'topic_id': 2,
  'dialogue_id': 0},
 {'key_words': ['Benefits', 'Parents', 'Survivor', 'Retirement', 'Support'],
  'topic_id': 3,
  'dialogue_id': 0},
 {'key_words': ['Disability benefits',
   'Application process',
   'Alternative methods',
   'Deafness',
   'Social Security office'],
  'topic_id': 0,
  'dialogue_id': 1},
 {'key_words': ['Social Security number',
   'Proof of age',
   'Doctor contacts',
   'Hospital information',
   'Medication details'],
  'topic_id': 1,
  'dialogue_id': 1},
 {'key_words': ['Apply',
   'Corrected card',
   'U',
   'Legally changed name',
   'Social Security'],
  'topic_id': 0,
  'dialogue_id': 2},
 {'key_

In [228]:
coh_metrics = []
exclude = set(string.punctuation)

for idx, i in enumerate(keywords):
    current_vocab = []
    cur_sent = [word.lower() for word in texts[idx]['text'].split() if word not in exclude]
    if len(cur_sent) != 0:
        current_vocab.append(cur_sent)
        
    i['topics'] = [[word.lower() for word in i['key_words'] if word != '']]
    if len(i['topics'][0]) == 0:
        coherence_score = 0
    else:
        try:
            coh_metric = Coherence(topk=5, measure='c_npmi', texts=current_vocab)
            coherence_score = coh_metric.score(i)
            if str(coherence_score) == 'nan':
                coherence_score = 0
        except:
            coherence_score = 0
            
    coh_metrics.append(coherence_score)

/home/timmiakov/repos/dynamic-topic-modeling/.venv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/timmiakov/repos/dynamic-topic-modeling/.venv/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [229]:
np.mean(coh_metrics)

-0.16266957964562273

In [231]:
metrics = []
exclude = set(string.punctuation)
topics_list = {'topics': []}
current_vocab = []

for idx, i in enumerate(keywords):   
    cur_sent = [word.lower() for word in texts[idx]['text'].split() if word not in exclude]
    if len(cur_sent) != 0:
        current_vocab.append(cur_sent)
    
    kw = [word.lower() for word in i['key_words'] if word != '']
    if len(kw) != 0:
        topics_list['topics'].append(kw)
            
cur_metrics = {"TopicDiversity": 0,
                    "Coherence": 0}
   
td_metric = TopicDiversity(topk=5) 
topic_diversity_score = td_metric.score(topics_list)

#coh_metric = Coherence(topk=5, measure='c_npmi', texts=current_vocab)
#coherence_score = coh_metric.score(topics_list)
    
cur_metrics = {"TopicDiversity": topic_diversity_score,
                "Coherence": coherence_score}
metrics.append(cur_metrics)
        

In [232]:
metrics

[{'TopicDiversity': 0.4074487895716946, 'Coherence': -0.414043645246712}]